In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# We need to define a function which defines a controlled U gate whose eigenvalue reflects the phase.

In [ ]:
def c_amod15(a, power):
    """
    """
    if a not in [2, 4, 7, 8, 11, 13]:
        raise ValueError("a should be in 2, 3, 7, 8, 11, 13")
    U = QuantumCircuit(4)
    for iteration in range(power)::
        if a in [2, 13]:
            U.swap(0,1)
            U.swap(1, 2)
            U.swap(2, 3)
        if a in [7,8]:
            U.swap(2, 3)
            U.swap(1,2)
            U.swap(0,1) 
        if a in [4, 11]:
            U.swap(1, 3)
            U.swap(0,2)
        if a in [7, 11, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15"%(a, power)
    c_U = U.control()
    return c_U

In [ ]:
n_count = 8
a = 7

In [ ]:
def qft_dagger(n):
    """"""
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT_d"
    return qc

In [ ]:
qc = QuantumCircuit(n_count + 4, n_count)
for q in range(n_count):
    qc.h(q)
qc.x(3 + n_count)

for q in range(n_count):
    qc.append(c_mod15(a, 2**q), [q] + [i+n_count for i in range(4)])
    
qc.append(qft_dagger(n_count), range(n_count))
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)

In [ ]:
aer_sim = Aer.get_backend("aer_simulator")
t_qc = transpile(qc, aer_sim)
qobj = assemble(t_qc)
results = aer_sim.run(qobj).result()
counts = results.get_count()
plot_histogram(counts)